In [1]:
from src.autoencoder import AutoEncoder
import pytorch_lightning as pl
from src.datamodule import CelebAData

In [2]:
import os
import pandas as pd
root = './data/celeba'
attributes = pd.read_csv(os.path.join(root, 'list_attr_celeba.csv'))
images_dir = os.path.join(root, 'img_align_celeba')


In [3]:
good_attribute_threshold = 0.3
good_attributes = ['image_id']
for key in attributes.keys():
    c = attributes[attributes[key]==1]['image_id'].count()
    if c < len(attributes)*(1-good_attribute_threshold) and c > len(attributes) * good_attribute_threshold:
        good_attributes.append(key)
gdf = attributes[good_attributes]

In [4]:
class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [5]:
list(attributes.keys()).index("Male")

21

In [6]:
args = Namespace(optimizer='adam', learning_rate=1e-3, weight_decay=1e-6,
    data_dir = './data/celeba',
    batch_size = 512,
    num_workers = 8,
    gpus=2
)
ae = AutoEncoder.load_from_checkpoint('logs/autoencoder/0114/200epochs/adam/version_6/checkpoints/epoch=187-step=15039.ckpt', hparams=args)
ae = ae.encoder

In [ ]:
from torch.utils.data import Dataset, DataLoader
from src.datamodule import CelebADataset
from torchvision import transforms as T

save_dir = "data_encoded"
ae.to("cuda:0")
transform = T.Compose(
[
    T.ToPILImage(),
    T.Resize((128, 128)),
    T.ToTensor(),
]
)
dataset = CelebADataset(root="data/celeba", train=0, transform=transform)

In [9]:
from datetime import datetime
from pytorch_lightning.loggers import TensorBoardLogger
logdir = 'logs/ppnet'
logdir += datetime.now().strftime("/%m%d")
logdir += '/{}epochs'.format(100)
logdir += '/{}'.format(args.optimizer)
logger = TensorBoardLogger(logdir, name='')

In [10]:
trainer = pl.Trainer.from_argparse_args(args,
    logger=logger,
    fast_dev_run=False,
    deterministic=True,
    enable_model_summary=False,
    log_every_n_steps=1,
)
dm = CelebAData(args)

trainer.fit(classifier, datamodule=dm
)

trainer.save_checkpoint(logger.log_dir+logger.name+"/"+logger.name+"celeba_test.ckpt")

/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:849: UserWarning: You requested multiple GPUs but did not specify a backend, e.g. `Trainer(strategy="dp"|"ddp"|"ddp2")`. Setting `strategy="ddp_spawn"` for you.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/2
initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=nccl
All distributed processes registered. Starting with 2 processes
----------------------------------------------------------------------------------------------------



Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch 0:   0%|          | 0/177 [00:00<?, ?it/s]                      

/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
Traceback (most recent call last):
  File "<string>", line 1, in <module>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/torch/__init__.py", line 721, in <module>
    import torch.utils.data
  File "/home/hjsong/anaconda3/envs/ppml/lib/python3.8/site-packages/torch/utils/data/__init__.py", line 38, in <module>
    from torch.utils.data.dataloader_expe